In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [10]:
with open('asv.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [12]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4576325


In [13]:
print(text[:1000])

American Standard Version

This Bible is in the Public Domain.


Genesis 1:1 In the beginning God created the heavens and the earth.
Genesis 1:2 And the earth was waste and void; and darkness was upon the face of the deep: and the Spirit of God moved upon the face of the waters.
Genesis 1:3 And God said, Let there be light: and there was light.
Genesis 1:4 And God saw the light, that it was good: and God divided the light from the darkness.
Genesis 1:5 And God called the light Day, and the darkness he called Night. And there was evening and there was morning, one day.
Genesis 1:6 And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.
Genesis 1:7 And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so.
Genesis 1:8 And God called the firmament Heaven. And there was evening and there was morning, a second day.
Genesis 1:9 And God said, Let the w

In [14]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWYZ[]abcdefghijklmnopqrstuvwxyz{}
82


In [16]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("Hi there")))

[61, 62, 1, 73, 61, 58, 71, 58]
Hi there


In [17]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([4576325]) torch.int64
tensor([27, 66, 58, 71, 62, 56, 54, 67,  1, 45, 73, 54, 67, 57, 54, 71, 57,  1,
        48, 58, 71, 72, 62, 68, 67,  0,  0, 46, 61, 62, 72,  1, 28, 62, 55, 65,
        58,  1, 62, 72,  1, 62, 67,  1, 73, 61, 58,  1, 42, 74, 55, 65, 62, 56,
         1, 30, 68, 66, 54, 62, 67,  9,  0,  0,  0, 33, 58, 67, 58, 72, 62, 72,
         1, 12, 21, 12,  1, 35, 67,  1, 73, 61, 58,  1, 55, 58, 60, 62, 67, 67,
        62, 67, 60,  1, 33, 68, 57,  1, 56, 71, 58, 54, 73, 58, 57,  1, 73, 61,
        58,  1, 61, 58, 54, 75, 58, 67, 72,  1, 54, 67, 57,  1, 73, 61, 58,  1,
        58, 54, 71, 73, 61,  9,  0, 33, 58, 67, 58, 72, 62, 72,  1, 12, 21, 13,
         1, 27, 67, 57,  1, 73, 61, 58,  1, 58, 54, 71, 73, 61,  1, 76, 54, 72,
         1, 76, 54, 72, 73, 58,  1, 54, 67, 57,  1, 75, 68, 62, 57, 22,  1, 54,
        67, 57,  1, 57, 54, 71, 64, 67, 58, 72, 72,  1, 76, 54, 72,  1, 74, 69,
        68, 67,  1, 73, 61, 58,  1, 59, 54, 56, 58,  1, 68, 59,  1, 73, 61, 58,
      

In [18]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [19]:
block_size = 8
train_data[:block_size+1]

tensor([27, 66, 58, 71, 62, 56, 54, 67,  1])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([27]) the target is: 66
when input is tensor([27, 66]) the target is: 58
when input is tensor([27, 66, 58]) the target is: 71
when input is tensor([27, 66, 58, 71]) the target is: 62
when input is tensor([27, 66, 58, 71, 62]) the target is: 56
when input is tensor([27, 66, 58, 71, 62, 56]) the target is: 54
when input is tensor([27, 66, 58, 71, 62, 56, 54]) the target is: 67
when input is tensor([27, 66, 58, 71, 62, 56, 54, 67]) the target is: 1


In [21]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data =train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)


print('------')


for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is: {target}")


inputs:
torch.Size([4, 8])
tensor([[71,  7,  1, 37, 62, 72, 61, 62],
        [54, 65, 65,  1, 65, 62, 75, 62],
        [ 1, 73, 61, 58, 62, 71,  1, 61],
        [74, 71, 67, 58, 71,  7,  1, 73]])
targets:
torch.Size([4, 8])
tensor([[ 7,  1, 37, 62, 72, 61, 62, 68],
        [65, 65,  1, 65, 62, 75, 62, 67],
        [73, 61, 58, 62, 71,  1, 61, 54],
        [71, 67, 58, 71,  7,  1, 73, 61]])
------
when input is [71] the target is: 7
when input is [71, 7] the target is: 1
when input is [71, 7, 1] the target is: 37
when input is [71, 7, 1, 37] the target is: 62
when input is [71, 7, 1, 37, 62] the target is: 72
when input is [71, 7, 1, 37, 62, 72] the target is: 61
when input is [71, 7, 1, 37, 62, 72, 61] the target is: 62
when input is [71, 7, 1, 37, 62, 72, 61, 62] the target is: 68
when input is [54] the target is: 65
when input is [54, 65] the target is: 65
when input is [54, 65, 65] the target is: 1
when input is [54, 65, 65, 1] the target is: 65
when input is [54, 65, 65, 1, 65] the

In [29]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
out = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 82])
tensor(4.8339, grad_fn=<NllLossBackward0>)

iB0{iGk4'(04W-8iZ-Vb[{aaQC/Ob0Bq:H:LiqETCqkiewJ<zsY
SCp'{=iCTap81k);3
5><k]:Hn,6tCCPUt1
snw>91R{.sgm


In [30]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

c:\Users\corne\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.317469835281372


In [41]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Psovathe to wsm, he cafrnghind bayot wevid men th Jodes t fin gatago d ge Th whe se cl al vert thoms he thine mis I os witoys.
Nur t, s wa be chucofigoves, intig aros ar.
Norempo w tthahe helethothe Jahe, wothap?
Is bodellld icin othousthemy uruthand tthedroveis Maundnd se m, Goviousonerovit-golthe 


In [26]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [24]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]  
        xbow[b,t] = torch.mean(xprev, 0)

In [32]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

False

In [71]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) 
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) 
q = query(x)

wei = q @ k.transpose(-2, -1) * (head_size ** -0.5)

tril = torch.tril(torch.ones(T,T))  
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)


v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [70]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [14]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
